# Final Project

Data Science pipeline:

Data Collection -> Data Processing -> Exploratory analysis + data visualization -> Analysis, hypothesis testing, & ML -> Insight & Policy Decision

## Part 1: Data Collection

First we gather our Steam Web API key a locally stored text document. A key can be requested from https://steamcommunity.com/dev/apikey.

In [1]:
key = open('key.txt').read()

Next we'll import a few key libraries that we will make use of during data collection.

In [2]:
import requests
import pandas as pd
import numpy as np

Now we can begin collecting data by first obtaining a list of every app on Steam.

In [3]:
URL = "http://api.steampowered.com/ISteamApps/GetAppList/v2"
PARAMS = {'key': key}
re = requests.get(url = URL, params = PARAMS)
apps = re.json()['applist']['apps']

-------- Eventually remove this next part --------

In [4]:
apps = apps[0:100]

Now that we have a list of all the apps, we can gather further information on them.

First, we define a list of all the categories of information that we want to know about each app.

In [5]:
categories = [
    ['name'],
    ['type'],
    ['steam_appid'],
    ['developers'],
    ['publishers'],
    ['is_free'],
    ['price_overview', 'initial'],
    ['achievements', 'total'],
    ['release_date', 'date'],
    ['metacritic', 'score']
]

We also need to create a filter parameter for our API requests so that we only retrieve the categories we defined above.

In [6]:
filters = ''
for cat in categories:
    filters += cat[0] + ','
filters += 'basic,genres'

Next, we create a function that will extract the desired information from each response we get from the requests.

In [7]:
def fetch(game_info, categories):
    info = {}
    for category in categories:
        cur_info = game_info
        failed = False
        
        for subpart in category:
            if not failed and subpart in cur_info.keys():
                cur_info = cur_info[subpart]
            else:
                failed = True
        
        if not failed:
            info[' '.join(category)] = cur_info
        else:
            info[' '.join(category)] = np.NaN
    
    # Genre is handled differently from the rest because a game can have multiple genres.
    # They can also have multiple publishers/developers but I don't think that we should
    # treat that the same way, because the numbers of genres should be much less than the
    # numbers of developers.
    # Not sure how the "dummy variable trap" works here either. I think each row should
    # have a column for each genre, but it doesn't make sense for the 0 vector to default
    # to some genre when games can have more than one. For example, if the 0 vector defaulted
    # to FPS, then for a game that is an FPS and RPG, what columns should be marked 1? If
    # it is just the RPG column, then how would you tell which games are just RPGs and which
    # ones are FPS RPG hybrids.
    # Also could use 'categories' instead of 'genres', but 'categories' has a bunch of extra
    # stuff like if the game has steam achievements etc.
    # Could also do both, but that would be a lot of columns (probably will be a bunch with
    # just genres)
    if 'genres' in game_info.keys():
        for genre in game_info['genres']:
            info['Genre: ' + genre['description']] = True
    
    
    return info

Finally we can collect data about each game.



For the finished project we'll probably have to do this process more than once, storing the resulting dataframe for smaller subsets of the full list. Then we'll combine them and store that dataframe, and load it in here. But we still need to mention the code we used to collect the data for the sake of replicability.

In [8]:
all_info = []

for app in apps:
    app_id = app['appid']
    # make main api query
    URL = "http://store.steampowered.com/api/appdetails"
    PARAMS = {'appids': app_id, 'filters': filters}
    re = requests.get(url = URL, params = PARAMS)
    if re.json()[str(app_id)]['success']:
        game_info = re.json()[str(app_id)]['data']
        info = fetch(game_info, categories)
        # make review api query
        URL = f"http://store.steampowered.com/appreviews/{app_id}?json=1"
        PARAMS = {'num_per_page': 0, 'language': 'all', 'purchase_type': 'all'}
        re = requests.get(url = URL, params = PARAMS)
        summary = re.json()
        if 'query_summary' in summary.keys():
            q_summary = summary['query_summary']

            if 'total_positive' in q_summary:
                info['total_positive'] = q_summary['total_positive']
            else:
                info['total_positive'] = np.NaN

            if 'total_reviews' in q_summary:
                info['total_reviews'] = q_summary['total_reviews']
            else:
                info['total_reviews'] = np.NaN


        all_info.append(info)

With the data collected, we can now turn it into a pandas dataframe.

I'm slightly concerned about how long it took it to do just 100 (probably like a minute). Might have to use a library like grequests to make multiple requests at once. I think that it is also possible to request information for multiple apps at once if we modify the first main api PARAMS to have more than one app_id for appids. But I'm not sure if the review API has something similar.

In [9]:
df = pd.DataFrame.from_dict(all_info)
df.head()

,name,type,steam_appid,developers,publishers,is_free,price_overview initial,achievements total,release_date date,metacritic score,...,Genre: Adventure,Genre: Free to Play,Genre: RPG,Genre: Massively Multiplayer,Genre: Design & Illustration,Genre: Web Publishing,Genre: Racing,Genre: Audio Production,Genre: Education,Genre: Utilities
0,Trine 4: Melody of Mystery Soundtrack,music,1503550,[Frozenbyte],[],False,699.0,NaN,"Dec 23, 2020",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Rift World Demo,demo,1503570,[Cerulean Interactive],[Cerulean Interactive],True,NaN,NaN,"Apr 13, 2021",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Epic City Builder 4,game,1503580,[Andrew Rowe],[Andrew Rowe],False,1999.0,19.0,"Oct 14, 2021",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Range is HOT!,game,1503590,[Winter Bear Studio],[Winter Bear Studio],False,2999.0,NaN,"Mar 9, 2021",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arm of Revenge Re-Edition,game,1503630,[MagicWing],[MagicWing],False,499.0,32.0,"Jan 14, 2021",NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Part 2: Data Processing

tbh I don't remember if data tidying is data processing, but if not we just move this stuff up

The dataframe now as all the information we want, but not all of it is in the desired format. First we replace NaN values from the genre columns with false. Values in the genres column are set to NaN if the app was not tagged as part of that genre, so this is a logical replacement value.

In [10]:
for col_name in df:
    if col_name.startswith('Genre: '):
        df[col_name].fillna(False, inplace=True)

Next we change the release dates to all be in the same format, the pandas datetime format. The parameter `errors='coerce'` means that if pandas cannot figure out what the date format means, then it will set the value to pd.NaT. This is reasonable because there are many apps and not all will use a recognizable format, but we do not want those few exceptions to bring the process to a complete stop. The main reason the release date would not be in a format recognized by pandas would be if the date is given in a language other than English.

In [11]:
df['release_date date'] = df['release_date date'].apply(lambda x: pd.to_datetime(x, errors='coerce'))

Now we define a function to determine positive review percentage for a row. Gives NaN if missing either the total review count or total count of positive reviews.

In [12]:
def positive_perc(row):
    if not np.isnan(row['total_reviews']) and row['total_reviews'] != 0 and not np.isnan(row['total_positive']):
        return row['total_positive'] / row['total_reviews']
    else:
        return np.NaN

With the function defined, we can now add a positive review percentage column to the dataframe.

In [13]:
df['positive_percentage'] = df.apply(positive_perc, axis=1)

We can see that for free games, the price column tends to be NaN. It is reasonable to say that if a game is free, then its price should be 0. We will now define a function to do this for us.

In [14]:
def get_price(row):
    if np.isnan(row['price_overview initial']) and row['is_free']:
        return 0
    else:
        return row['price_overview initial']

We apply this function to the price_overview initial column, and rename the column to just price for the sake of brevity.

In [15]:
df['price_overview initial'] = df.apply(get_price, axis=1)
df.rename({'price_overview initial': 'price'}, axis='columns', inplace=True)

Seems reasonable to replace NaN achievements total with 0

In [16]:
df['achievements total'].fillna(0, inplace=True)

Now let's get an idea of what the tidied up dataframe looks like

In [17]:
df.head()

,name,type,steam_appid,developers,publishers,is_free,price,achievements total,release_date date,metacritic score,...,Genre: Free to Play,Genre: RPG,Genre: Massively Multiplayer,Genre: Design & Illustration,Genre: Web Publishing,Genre: Racing,Genre: Audio Production,Genre: Education,Genre: Utilities,positive_percentage
0,Trine 4: Melody of Mystery Soundtrack,music,1503550,[Frozenbyte],[],False,699.0,0.0,2020-12-23,NaN,...,False,False,False,False,False,False,False,False,False,1.000000
1,Rift World Demo,demo,1503570,[Cerulean Interactive],[Cerulean Interactive],True,0.0,0.0,2021-04-13,NaN,...,False,False,False,False,False,False,False,False,False,NaN
2,Epic City Builder 4,game,1503580,[Andrew Rowe],[Andrew Rowe],False,1999.0,19.0,2021-10-14,NaN,...,False,False,False,False,False,False,False,False,False,0.833333
3,Range is HOT!,game,1503590,[Winter Bear Studio],[Winter Bear Studio],False,2999.0,0.0,2021-03-09,NaN,...,False,False,False,False,False,False,False,False,False,0.743590
4,Arm of Revenge Re-Edition,game,1503630,[MagicWing],[MagicWing],False,499.0,32.0,2021-01-14,NaN,...,False,False,False,False,False,False,False,False,False,0.772727
